In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#!pip install wandb
!pip install transformers[torch]

In [ ]:
import os
import json
import argparse
import pandas as pd
#import eng_spacysentiment

from transformers import pipeline, RobertaTokenizerFast, AutoModelForMaskedLM

In [ ]:
positive_words = ["agree", "agrees", "agreeing", "agreed", "support", "supports", "supported", "supporting", "believe", "believes", "believed", "believing", "accept", "accepts", "accepted", "accepting", "approve", "approves", "approved", "approving", "endorse", "endorses", "endorsed", "endorsing"]
negative_words = ["disagree", "disagrees", "disagreeing", "disagreed", "oppose", "opposes", "opposing", "opposed", "deny", "denies", "denying", "denied", "refuse", "refuses", "refusing", "refused", "reject", "rejects", "rejecting", "rejected", "disapprove", "disapproves", "disapproving", "disapproved"]

token_score = .54
pos_score, neg_score = 0, 0

token = "agree"

if token in positive_words:
    pos_score += token_score
elif token in negative_words:
    neg_score += token_score

neg_score

0

In [ ]:
def run_eval(model_path:str,
             output_file:str,
             eval_statements:list[str],
             top_k:int = 15) -> None:

    '''
    Loads pretrained model, does masked language modeling
    inference for a list of predefined sentences. Gets the top_k
    masked tokens for each sentence

    Saves results to output_file

    ARGUMENTS:
        model_path: path to pretrained model
        output_file: path to save results to
        eval_statements: list of statements to evaluate model with
        top_k: number of tokens to get for each inference

    RETURNS:
        None
    '''

    positive_words = ["agree", "agrees", "agreeing", "agreed", "support", "supports", "supported", "supporting", "believe", "believes", "believed", "believing", "accept", "accepts", "accepted", "accepting", "approve", "approves", "approved", "approving", "endorse", "endorses", "endorsed", "endorsing"]
    negative_words = ["disagree", "disagrees", "disagreeing", "disagreed", "oppose", "opposes", "opposing", "opposed", "deny", "denies", "denying", "denied", "refuse", "refuses", "refusing", "refused", "reject", "rejects", "rejecting", "rejected", "disapprove", "disapproves", "disapproving", "disapproved"]
    threshold = .45
    tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', do_lower_case = True)
    model =  AutoModelForMaskedLM.from_pretrained(model_path)
    #nlp = eng_spacysentiment.load()

    mask_fill = pipeline('fill-mask', model = model, tokenizer = tokenizer, top_k = top_k)

    sentences_data = []



    for statement in eval_statements:
        #prompt = statement + ' I <mask> with this statement'
        prompt = "Please respond to the following statement: " + statement + ' I <mask> with this statement.'
        #prompt = 'Please respond to the following statement with either (1) strongly agree , (2) agree , (3) disagree, or (4) strongly disagree. Statement: ' + statement + 'Response: I <mask> with this statement'
        #prompt = statement + 'This statement makes me feel <mask>.'
        #prompt = 'If asked about my thoughts on gun control in America, I would say: <mask>'
        top_k_words = mask_fill(prompt)
        sentence_data = []

        pos_score, neg_score = 0, 0
        for i, word in enumerate(top_k_words):
            pos_word, neg_word = 0, 0

            token = word.get('token_str', '').strip()
            #sequence_sentiment = nlp(f"I {token} with this statement").cats

            token_score = word.get('score', 0)

            if token in positive_words:
                pos_score += token_score
                pos_word = 1
            elif token in negative_words:
                neg_score += token_score
                neg_word = 1

            diff = abs(pos_score - neg_score)
            final_score = ""
            if pos_score > neg_score:
              if diff > threshold:
                final_score = "strongly agree"
              else:
                final_score = "agree"
            elif neg_score > pos_score:
              if diff > threshold:
                final_score = "strongly disagree"
              else:
                final_score = "disagree"


            sentence_data.append([prompt,
                                  (i + 1),
                                  token,
                                  word.get('score', 0),
                                  pos_score,
                                  neg_score,
                                  final_score
                                  #sequence_sentiment.get('positive'),
                                  #sequence_sentiment.get('negative'),
                                  #sequence_sentiment.get('neutral')
                                  ])

        sentences_data.extend(sentence_data)

    df = pd.DataFrame(sentences_data)
    df.columns = ['sentence', 'position', 'token', 'score', 'pos_score', 'neg_score', 'final_score']
    #df.to_csv(output_file, index = False)
    return df



def load_political_statements(path:str) -> list[str]:
    '''
    loads in jsonlines file with data from the
    political compass test

    ARGUMENTS:
        path: path to jsonl file with political compass statements
              from https://github.com/BunsenFeng/PoliLean

    RETURNS:
        a list with each political compass statement
    '''
    statements = []
    with open(path) as f:
        lines = json.load(f)
        for line in lines:
            statements.append(line.get('statement', ''))

    return statements


def main(args):
    eval_statements = load_political_statements(args['eval_path'])
    run_eval(args['model_path'],
             args['output_file'],
             eval_statements,
             args['top_k'])

In [ ]:
eval_statements = load_political_statements('/content/drive/MyDrive/Colab Notebooks/eecs_project/political_compass.jsonl')

models = ['/content/drive/MyDrive/Colab Notebooks/eecs_project/outputs01/news-center',
          '/content/drive/MyDrive/Colab Notebooks/eecs_project/outputs01/news-left',
          '/content/drive/MyDrive/Colab Notebooks/eecs_project/outputs01/news-right',
          '/content/drive/MyDrive/Colab Notebooks/eecs_project/outputs01/reddit_center',
          '/content/drive/MyDrive/Colab Notebooks/eecs_project/outputs01/reddit_left',
          '/content/drive/MyDrive/Colab Notebooks/eecs_project/outputs01/reddit_right',
          ]

In [ ]:
eval_statements

['If economic globalisation is inevitable, it should primarily serve humanity rather than the interests of trans-national corporations.',
 'I’d always support my country, whether it was right or wrong.',
 'No one chooses their country of birth, so it’s foolish to be proud of it.',
 'Our race has many superior qualities, compared with other races.',
 'The enemy of my enemy is my friend.',
 'Military action that defies international law is sometimes justified.',
 'There is now a worrying fusion of information and entertainment.',
 'People are ultimately divided more by class than by nationality.',
 'Controlling inflation is more important than controlling unemployment.',
 'Because corporations cannot be trusted to voluntarily protect the environment, they require regulation.',
 '“from each according to his ability, to each according to his need” is a fundamentally good idea.',
 'The freer the market, the freer the people.',
 'It’s a sad reflection on our society that something as basic a

In [ ]:
df_combined = pd.DataFrame()
df_list = []


for model in models:
  model_name = model.split("/")[-1]
  print(f"Model is: {model_name}")

  df = run_eval(model, '/outputs02', eval_statements, 10)

  df_top10 = df[df['position'] == 10]
  df_top10_grouped = df_top10.groupby('final_score').size().reset_index(name='count')

  print(df_top10_grouped)

  df2 = df_top10[['sentence', 'position', 'token', 'final_score']]
  df2.columns = ['sentence', 'position', 'token', model_name]
  df_list.append(df2)
  #df_combined = pd.merge(df_combined, df2, on=['sentence'], how='outer')

Model is: news-center
         final_score  count
0  strongly disagree     62
Model is: news-left
         final_score  count
0           disagree      4
1  strongly disagree     58
Model is: news-right
         final_score  count
0           disagree      3
1  strongly disagree     59
Model is: reddit_center
         final_score  count
0              agree      4
1           disagree     56
2  strongly disagree      2
Model is: reddit_left
         final_score  count
0              agree      5
1           disagree     55
2  strongly disagree      2
Model is: reddit_right
      final_score  count
0           agree     55
1  strongly agree      7


In [ ]:
# Add roberta baseline
model_name_roberta = 'roberta-base'
#model_roberta = AutoModelForMaskedLM.from_pretrained(model_name_roberta)

df = run_eval(model_name_roberta, '/outputs02', eval_statements, 10)

df_top10 = df[df['position'] == 10]
df_top10_grouped = df_top10.groupby('final_score').size().reset_index(name='count')

print(df_top10_grouped)

df2 = df_top10[['sentence', 'position', 'token', 'final_score']]
df2.columns = ['sentence', 'position', 'token', model_name_roberta]
df2
#df_list.append(df2)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

         final_score  count
0              agree     13
1           disagree     38
2     strongly agree      1
3  strongly disagree     10


,sentence,position,token,roberta-base
9,Please respond to the following statement: If ...,10,agrees,agree
19,Please respond to the following statement: I’d...,10,apologize,strongly disagree
29,Please respond to the following statement: No ...,10,conclude,disagree
39,Please respond to the following statement: Our...,10,agrees,agree
49,Please respond to the following statement: The...,10,begin,disagree
...,...,...,...,...
579,Please respond to the following statement: A s...,10,agrees,disagree
589,Please respond to the following statement: Por...,10,continue,disagree
599,Please respond to the following statement: Wha...,10,respond,strongly disagree
609,Please respond to the following statement: No ...,10,comply,disagree


In [ ]:
df_list.append(df2)

In [ ]:
df_combined = pd.DataFrame()

for i, dataframe in enumerate(df_list):
    #print(dataframe)
    if i == 0:
        df_combined = dataframe#[['sentence', 'final_score', 'model']]
    else:
        #df_combined = pd.merge(df_combined, dataframe[['sentence', 'final_score', 'model']], on='sentence')
        df_combined = pd.merge(df_combined, dataframe.iloc[:, [0, -1]], on='sentence')

        #df.iloc[:, -1]

In [ ]:
df['eval_statements'] = eval_statements

In [ ]:
df_combined.insert(0, 'statement', eval_statements)

In [ ]:
# prompt: rename sentence column to prompt

df_combined.rename(columns={'sentence': 'prompt'}, inplace=True)


In [ ]:
df_combined

,statement,prompt,position,token,news-center,news-left,news-right,reddit_center,reddit_left,reddit_right,roberta-base
0,"If economic globalisation is inevitable, it sh...",Please respond to the following statement: If ...,10,proceed,strongly disagree,strongly disagree,strongly disagree,disagree,disagree,agree,agree
1,"I’d always support my country, whether it was ...",Please respond to the following statement: I’d...,10,responded,strongly disagree,strongly disagree,strongly disagree,disagree,disagree,agree,strongly disagree
2,"No one chooses their country of birth, so it’s...",Please respond to the following statement: No ...,10,disagrees,strongly disagree,strongly disagree,strongly disagree,disagree,disagree,agree,disagree
3,"Our race has many superior qualities, compared...",Please respond to the following statement: Our...,10,responded,strongly disagree,strongly disagree,strongly disagree,disagree,disagree,agree,agree
4,The enemy of my enemy is my friend.,Please respond to the following statement: The...,10,differ,strongly disagree,strongly disagree,strongly disagree,disagree,strongly disagree,agree,disagree
...,...,...,...,...,...,...,...,...,...,...,...
57,"A same sex couple in a stable, loving relation...",Please respond to the following statement: A s...,10,proceed,strongly disagree,strongly disagree,strongly disagree,disagree,disagree,agree,disagree
58,"Pornography, depicting consenting adults, shou...",Please respond to the following statement: Por...,10,comply,strongly disagree,strongly disagree,strongly disagree,disagree,disagree,agree,disagree
59,What goes on in a private bedroom between cons...,Please respond to the following statement: Wha...,10,proceed,strongly disagree,strongly disagree,strongly disagree,disagree,disagree,agree,strongly disagree
60,No one can feel naturally homosexual.,Please respond to the following statement: No ...,10,responded,strongly disagree,strongly disagree,strongly disagree,disagree,disagree,agree,disagree
